In [37]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import glob


In [38]:


# Load annotations
annotation_path = r"C:\Users\nivet\OneDrive\Desktop\New folder\Regression\asssolution\Licplatesdetection_train.csv"
annotations = pd.read_csv(r"C:\Users\nivet\OneDrive\Desktop\New folder\Regression\asssolution\Licplatesdetection_train.csv")

# Load images and corresponding bounding boxes
image1 = []
bounding_boxes = []
target_size = (128, 128)
folder_path = r"C:\Users\nivet\OneDrive\Desktop\New folder\Regression\asssolution\license_plates_detection_train"

for index, row in annotations.iterrows():
    try:
        img_path = os.path.join(folder_path, row['img_id'])
        img = Image.open(img_path)
        img = img.resize(target_size)
        img = np.asarray(img)
        image1.append(img)
        
        # Append bounding box as [xmin, ymin, xmax, ymax]
        bounding_boxes.append([row['xmin'], row['ymin'], row['xmax'], row['ymax']])
    except Exception as e:
        print(f"Error processing file {row['img_id']}: {e}")

image1 = np.array(image1)
bounding_boxes = np.array(bounding_boxes)
print(f"Loaded {len(image1)} images and {len(bounding_boxes)} bounding boxes.")



Loaded 900 images and 900 bounding boxes.


In [39]:
# Initialize lists for storing image data and text labels
image2 = []
labels2 = []  
target_size = (128, 128)  

# Folder path for images of license plates with text annotations
folder_path2 = r"C:\Users\nivet\OneDrive\Desktop\New folder\Regression\asssolution\license_plates_recognition_train"

# Loop through each image file in the folder
for f in glob.iglob(folder_path2 + '/*'):
    try:
        # Open the image
        img = Image.open(f)
        img = img.resize(target_size)
        img = np.asarray(img)  
        image2.append(img)

        # Extract the alphanumeric label from the filename (e.g., 'ABC1234.jpg')
        filename = os.path.basename(f)
        label = filename.split('.')[0]  
        labels2.append(label)

    except Exception as e:
        print(f"Error processing file {f}: {e}")

# Convert lists to numpy arrays
image2 = np.array(image2)
labels2 = np.array(labels2)
print(f"Loaded {len(image2)} images and {len(labels2)} labels.")


Error processing file C:\Users\nivet\OneDrive\Desktop\New folder\Regression\asssolution\license_plates_recognition_train\model.ipynb: cannot identify image file 'C:\\Users\\nivet\\OneDrive\\Desktop\\New folder\\Regression\\asssolution\\license_plates_recognition_train\\model.ipynb'
Loaded 900 images and 900 labels.


In [40]:

import tensorflow as tf

In [41]:
print(f"Processed images shape: {image1.shape}")
print(f"Processed labels shape: {bounding_boxes.shape}")
print(f"Processed images shape: {image2.shape}")
print(f"Processed labels shape: {label2.shape}")

Processed images shape: (900, 128, 128, 3)
Processed labels shape: (900, 4)
Processed images shape: (900, 128, 128, 3)
Processed labels shape: (900,)


In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
detection_model=Sequential()
# Add Convolutional layers and MaxPooling layers
detection_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
detection_model.add(MaxPooling2D((2, 2)))
detection_model.add(Conv2D(64, (3, 3), activation='relu'))
detection_model.add(MaxPooling2D((2, 2)))

# Flatten the features and add Dense layers
detection_model.add(Flatten())
detection_model.add(Dense(128, activation='relu'))
# Output layer for bounding box prediction (xmin, ymin, xmax, ymax)
detection_model.add(Dense(4, activation='linear'))

# Compile the model
detection_model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['accuracy'])

# Display model summary
detection_model.summary()

# Train the model on the license plate detection dataset
detection_model.fit(image1, bounding_boxes, batch_size=32, epochs=20)

c:\Users\nivet\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │     7,372,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,392,836 (28.20 MB)

 Trainable params: 7,392,836 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 7s 175ms/step - accuracy: 0.4758 - loss: 895328.9375
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 175ms/step - accuracy: 0.4484 - loss: 17576.1426
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 179ms/step - accuracy: 0.5916 - loss: 14383.3301
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 183ms/step - accuracy: 0.6399 - loss: 12132.0195
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 179ms/step - accuracy: 0.7087 - loss: 11545.9092
Epoch 6/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 179ms/step - accuracy: 0.7607 - loss: 10043.4199
Epoch 7/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 169ms/step - accuracy: 0.7623 - loss: 8874.4072
Epoch 8/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 166ms/step - accuracy: 0.7824 - loss: 8216.8330
Epoch 9/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 161ms/step - accuracy: 0.8104 - loss: 6438.0669
Epoch 10/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 163ms/step - accuracy: 0.8210 - loss: 6172.0190
Epoch 11/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 163ms/step - accuracy: 0.8512 - loss: 4587.1592
Epoch 12/20


In [43]:

# Train the detection model
detection_model.fit(image1, bounding_boxes, batch_size=32, epochs=20)


Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 172ms/step - accuracy: 0.9397 - loss: 1028.0378
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 174ms/step - accuracy: 0.9416 - loss: 953.9871
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 167ms/step - accuracy: 0.9457 - loss: 819.9360
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 169ms/step - accuracy: 0.9337 - loss: 736.0030
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 168ms/step - accuracy: 0.9488 - loss: 735.9661
Epoch 6/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 170ms/step - accuracy: 0.9424 - loss: 620.5280
Epoch 7/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 168ms/step - accuracy: 0.9284 - loss: 706.2938
Epoch 8/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 169ms/step - accuracy: 0.9538 - loss: 575.7527
Epoch 9/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 168ms/step - accuracy: 0.9542 - loss: 844.8872
Epoch 10/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 170ms/step - accuracy: 0.9560 - loss: 844.0099
Epoch 11/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 168ms/step - accuracy: 0.9337 - loss: 733.8171
Epoch 12/20
29/29 ━━━━━━━━━━━

In [100]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape
from tensorflow.keras.optimizers import Adam

num_classes = 36  
sequence_length = 8  

recognition_model = Sequential()

# Convolutional layers
recognition_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
recognition_model.add(MaxPooling2D((2, 2)))
recognition_model.add(Conv2D(64, (3, 3), activation='relu'))
recognition_model.add(MaxPooling2D((2, 2)))
recognition_model.add(Flatten())

# Dense layer
recognition_model.add(Dense(128, activation='relu'))

# Output layer reshaped for sequence prediction (sequence_length x num_classes)
recognition_model.add(Dense(8 * 36, activation='softmax'))  
recognition_model.add(Reshape((8, 36))) 

recognition_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Display model summary
recognition_model.summary()







Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_32 (Conv2D)              │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_32 (MaxPooling2D) │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_33 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_17 (Flatten)            │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 128)            │     7,372,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 288)            │        37,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_13 (Reshape)            │ (None, 8, 36)          │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,429,472 (28.34 MB)

 Trainable params: 7,429,472 (28.34 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Character-to-integer mapping for alphanumeric characters
char_to_int = {char: idx for idx, char in enumerate("0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ")}
int_to_char = {idx: char for char, idx in char_to_int.items()}

# Sample labels 
labels2 = ["ABC123", "9X2YZ", "A1B2C3", "XYZ789", "QWE123", "Z9Y456"]  

# Convert the text labels into integer sequences
labels2_int = [[char_to_int[char] for char in label] for label in labels2]

# Pad the sequences to a fixed length of 8 
labels2_int_padded = pad_sequences(labels2_int, maxlen=8, padding='post')

# One-hot encode the labels
labels2_onehot = np.array([np.eye(36)[label] for label in labels2_int_padded]) 







In [102]:


image_data = np.random.rand(len(labels2), 128, 128, 3)  

# Train the model
recognition_model.fit(image_data, labels2_onehot, batch_size=32, epochs=20)





Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0208 - loss: 3.5239
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.3333 - loss: 4.2726
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.3750 - loss: 2.7591
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.4583 - loss: 2.1930
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.5417 - loss: 1.7812
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.5833 - loss: 1.6187
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - accuracy: 0.6667 - loss: 1.4446
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.7292 - loss: 1.2243
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.7917 - loss: 1.0528
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 0.7917 - loss: 0.8767
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.8542 - loss: 0.7087
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.9375 - loss:

In [103]:
from PIL import Image
import numpy as np
import glob

# Load test images
test_folder_path = r"C:\Users\nivet\OneDrive\Desktop\New folder\Regression\asssolution\test\test" 
test_images = []
target_size = (128, 128)

for image_path in glob.iglob(test_folder_path + '/*.jpg'):
    try:
        img = Image.open(image_path).convert('RGB')
        img = img.resize(target_size)
        img = np.asarray(img)
        test_images.append(img)
    except Exception as e:
        print(f"Error processing file {image_path}: {e}")

test_images = np.array(test_images)
print(f"Loaded {len(test_images)} test images with shape {test_images.shape}.")


Loaded 210 test images with shape (210, 128, 128, 3).


In [104]:
from tensorflow.keras.layers import TimeDistributed, Dense

num_classes = 36  
sequence_length = 8

# Change the model output for multi-class sequences
recognition_model.add(Dense(num_classes * sequence_length, activation='softmax'))
recognition_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [105]:
# Dummy labels (one label per image) for sparse categorical loss
dummy_labels = np.zeros((len(test_images), sequence_length), dtype=int)



In [106]:
loss, accuracy = recognition_model.evaluate(test_images, dummy_labels, batch_size=32)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.8750 - loss: 5.6514
Test Loss: 5.6514
Test Accuracy: 0.8750
